In [1]:
import pandas as pd
# file name 합치기
from glob import glob
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from sklearn.model_selection import train_test_split
# 시각화
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from skimage import io, feature
from skimage.transform import resize
import cv2
from PIL import ImageChops

import seaborn as sns

# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

In [4]:
에너지사용 = pd.read_csv("../생활환경개선/데이터파일/서울특별시_자치구 법정동별 에너지사용량 통계_주거용_20230512.csv",encoding='EUC-KR')


In [12]:
에너지사용

,구분,자치구명,법정동명,건물유형,2022년 1월,2022년 2월,2022년 3월,2022년 4월,2022년 5월,2022년 6월,2022년 7월,2022년 8월,2022년 9월,2022년 10월,2022년 11월,2022년 12월
0,전기,강남구,개포동,공동주택,7200943,6617228,6493290,6029290,6479576,8591661,9211776.0,6948301.0,6987802.0,6252099.0,6360880.0,6841707.0
1,전기,강남구,개포동,단독주택,600037,518883,499469,438967,465250,584221,738246.0,612861.0,619261.0,480883.0,460891.0,497147.0
2,전기,강남구,논현동,공동주택,5549624,4889737,4822460,4459243,4948666,6424482,7498085.0,6111260.0,6159100.0,4891942.0,4637516.0,4992352.0
3,전기,강남구,논현동,단독주택,3252658,2753392,2719221,2516477,2866030,3760844,4468862.0,3541520.0,3548236.0,2744543.0,2561600.0,2813587.0
4,전기,강남구,대치동,공동주택,9672351,9024081,8574895,8287759,9229790,12421630,13281322.0,9977035.0,11198965.0,9848877.0,9762137.0,10872472.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,지역난방,은평구,진관동,공동주택,24066,17154,8972,5597,3580,2917,2764.0,NaN,3366.0,7629.0,12080.0,27135.0
1682,지역난방,중랑구,망우동,공동주택,1414,1188,674,429,290,230,208.0,NaN,277.0,540.0,825.0,1775.0
1683,지역난방,중랑구,묵동,공동주택,124,89,48,29,18,15,15.0,NaN,17.0,40.0,63.0,142.0
1684,지역난방,중랑구,상봉동,공동주택,1630,1211,653,381,221,182,173.0,NaN,262.0,607.0,873.0,1824.0


In [13]:
에너지사용['구분'].unique()

array(['전기', '가스', '지역난방'], dtype=object)

In [14]:
에너지사용['자치구명'].unique()

array(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object)

In [15]:
에너지사용['법정동명'].unique()

array(['개포동', '논현동', '대치동', '도곡동', '삼성동', '세곡동', '수서동', '신사동', '압구정동',
       '역삼동', '율현동', '일원동', '자곡동', '청담동', '강일동', '고덕동', '길동', '둔촌동',
       '명일동', '상일동', '성내동', '암사동', '천호동', '미아동', '번동', '수유동', '우이동',
       '가양동', '개화동', '공항동', '내발산동', '등촌동', '마곡동', '방화동', '염창동', '오곡동',
       '외발산동', '화곡동', '남현동', '봉천동', '신림동', '광장동', '구의동', '군자동', '능동',
       '자양동', '중곡동', '화양동', '가리봉동', '개봉동', '고척동', '구로동', '궁동', '신도림동',
       '오류동', '온수동', '천왕동', '항동', '가산동', '독산동', '시흥동', '공릉동', '상계동',
       '월계동', '중계동', '하계동', '도봉동', '방학동', '쌍문동', '창동', '답십리동', '신설동',
       '용두동', '이문동', '장안동', '전농동', '제기동', '청량리동', '회기동', '휘경동', '노량진동',
       '대방동', '동작동', '본동', '사당동', '상도1동', '상도동', '신대방동', '흑석동', '공덕동',
       '구수동', '노고산동', '당인동', '대흥동', '도화동', '동교동', '마포동', '망원동', '상수동',
       '상암동', '서교동', '성산동', '신공덕동', '신수동', '신정동', '아현동', '연남동', '염리동',
       '용강동', '중동', '창전동', '토정동', '하중동', '합정동', '현석동', '남가좌동', '냉천동',
       '대신동', '대현동', '미근동', '봉원동', '북가좌동', '북아현동', '신촌동', '연희동', '영천동',
       '옥천동',

In [16]:
에너지사용['건물유형'].unique()

array([' 공동주택 ', ' 단독주택 '], dtype=object)